📄 Dans cette page, je vais décrire le processus de nettoyage pour le sous-ensemble de données concernant **l'État de santé**

---

## I. Morbidité chronique auto-déclarée *(DataSet Groupe)* : Hamza OUADOUD

1. **Personnes avec un problème de santé ou une maladie de longue durée**,  
   par sexe, âge et statut au **regard de l'emploi**.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; **Importation des bibliothèques nécessaires  :**

In [1]:
import numpy as np
import pandas as pd
import eurostat
import random

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; **Inspection des données :**
* Affichage des données bruts

In [2]:
df = eurostat.get_data_df('hlth_silc_04')
df.sample(4)

,freq,unit,wstatus,age,sex,geo\TIME_PERIOD,2008,2009,2010,2011,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
9927,A,PC,POP,Y25-34,M,LT,15.1,9.1,5.8,7.4,...,7.9,8.8,7.9,8.1,11.5,10.9,15.9,12.6,12.4,10.5
9812,A,PC,POP,Y25-29,T,BE,10.6,11.6,10.7,11.4,...,11.3,13.1,12.0,12.2,10.8,10.0,9.4,11.3,13.7,13.0
15041,A,PC,SAL,Y45-64,M,EA19,NaN,27.1,27.8,27.2,...,29.9,33.6,34.3,36.5,35.5,34.6,32.3,32.4,NaN,NaN
2232,A,PC,EMP,Y_GE85,M,LT,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


* Description des données : 

In [ ]:
df.info()

* Problèmes rencontrés :
    - Présence de colonnes inutiles.
    - Lignes contenant des valeurs nulles.
    - Colonnes avec des types de données inappropriés.
    - Colonnes illisibles.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; **Nettoyage des données :**
* Sélection des colonnes utiles : 

In [4]:
df = df[['geo\\TIME_PERIOD','age','sex','wstatus','2019']]
df.head(4)

,geo\TIME_PERIOD,age,sex,wstatus,2019
0,AL,Y16-24,F,EMP,NaN
1,AT,Y16-24,F,EMP,10.4
2,BE,Y16-24,F,EMP,NaN
3,BG,Y16-24,F,EMP,NaN


* Renommage des colonnes :

In [5]:
df = df.rename(columns={
    'geo\\TIME_PERIOD':'Geo',
    'wstatus':'Activité_situation'
})
df.head(1)

,Geo,age,sex,Activité_situation,2019
0,AL,Y16-24,F,EMP,NaN


* Remplacement des codes pays par leur nom complet :

In [6]:
ds1_coutry_code = {
    "EU": "European Union (EU6-1958, EU9-1973, EU10-1981, EU12-1986, EU15-1995, EU25-2004, EU27-2007, EU28-2013, EU27-2020)",
    "EU27_2020": "European Union - 27 countries (from 2020)",
    "FR": "France",
    "HR": "Croatia",
    "IT": "Italy",
    "CY": "Cyprus",
    "LV": "Latvia",
    "LT": "Lithuania",
    "LU": "Luxembourg",
    "HU": "Hungary",
    "AL": "Albania",
    "RS": "Serbia",
    "TR": "Türkiye",
    "XK": "Kosovo*"
}

df['Geo'] = df['Geo'].replace(ds1_coutry_code)

* Remplacement des codes de sexe par leur nom complet :

In [7]:
for i in range(len(df)) : 
    if df.loc[i,'sex'] == 'M' : 
        df.loc[i,'sex'] = 'Male'
    elif df.loc[i,'sex'] == 'F' : 
        df.loc[i,'sex'] ='Female'
    else : 
        df.loc[i,'sex'] = 'Total'

* Remplacement des codes d'activité :


In [8]:
# Suppression des lignes où 'Activité_situation' contient 'EMP' ou 'INAC_OTH'
df = df[~df['Activité_situation'].isin(['EMP', 'INAC_OTH'])]

ds1_activity_code = {
    "POP": "Population",
    "SAL": "Salariés",
    "NSAL": "Personnes_occupées_sauf_salariés",
    "NEMP": "Personnes_non_occupées",
    "UNE": "Chômeurs",
    "RET": "Retraités",
}

# Remplacer les codes dans 'Activité_situation' par les descriptions complètes
df['Activité_situation'] = df['Activité_situation'].replace(ds1_activity_code)

* Remplissage des valeurs manquantes pour l'année 2018 :


In [ ]:
# Remplir les valeurs manquantes de l'année 2019 par les valeurs de l'année précédente (2018)
df['2019'] = df.groupby('Geo')['2019'].fillna(method='ffill')

# Garder uniquement les lignes avec des données valides pour 2019 en cas que toutes les années contient des valeur null
df = df[df['2019'].notna()]  

* Filtrer le DataFrame pour inclure uniquement les lignes avec 'TOTAL' pour sexe

In [10]:
df = df[df['sex'] =="Total"]

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; **Présentation des données final :**
* Affichage des données après nettoyage :

In [11]:
df.head(7)

,Geo,age,sex,Activité_situation,2019
4680,Albania,Y16-24,Total,Personnes_non_occupées,2.3
4681,Austria,Y16-24,Total,Personnes_non_occupées,22.6
4682,Belgium,Y16-24,Total,Personnes_non_occupées,10.6
4683,Bulgaria,Y16-24,Total,Personnes_non_occupées,3.7
4684,Switzerland,Y16-24,Total,Personnes_non_occupées,16.7
4685,Cyprus,Y16-24,Total,Personnes_non_occupées,9.0
4686,Czechia,Y16-24,Total,Personnes_non_occupées,9.9


* Pivotage des données :

In [12]:
# Création d'un tableau croisé dynamique à partir du DataFrame 'df'.
pivot_df = df.pivot_table(
    index=['Geo'],  
    columns='Activité_situation',  
    values='2019',  
).reset_index()

# Suppression des lignes contenant des valeurs manquantes (NaN) dans le DataFrame.
pivot_df = pivot_df.dropna()

# Arrondi des valeurs numériques du DataFrame à deux décimales.
pivot_df = pivot_df.round(2)

pivot_df.head(6)

Activité_situation,Geo,Chômeurs,Personnes_non_occupées,Personnes_occupées_sauf_salariés,Population,Retraités,Salariés
0,Albania,15.52,33.49,24.14,28.33,58.25,12.95
1,Austria,54.72,50.06,32.65,40.13,65.61,26.45
2,Belgium,35.54,37.41,20.48,26.99,38.24,18.02
3,Bulgaria,14.22,31.85,19.85,24.14,52.12,21.67
4,Croatia,37.78,46.99,32.42,38.96,74.52,22.07
5,Cyprus,50.74,55.69,56.42,46.76,83.20,36.65


**2.** **Personnes avec un problème de santé ou une maladie de longue durée**,  
   par sexe, âge et **d'éducation atteint**.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; **Inspection des données :**
* Affichage des données brutes :

In [14]:
df2 = eurostat.get_data_df('hlth_silc_05')
df2.sample(4)

,freq,unit,isced11,age,sex,geo\TIME_PERIOD,2008,2009,2010,2011,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
9472,A,PC_POP,ED0-2,Y_GE65,M,RS,NaN,NaN,NaN,NaN,...,58.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4380,A,PC,ED3_4,Y_GE65,M,IE,42.7,50.9,54.8,48.6,...,51.3,40.0,42.4,45.2,39.0,46.6,46.5,50.0,51.4,49.2
9121,A,PC,TOTAL,Y_GE65,T,CH,48.2,50.8,51.5,49.2,...,54.0,52.0,54.7,52.8,51.2,52.3,51.3,53.1,57.0,54.6
3006,A,PC,ED3_4,Y25-29,T,EU27_2020,NaN,NaN,13.5,14.3,...,18.3,19.1,18.7,21.6,20.9,20.3,NaN,NaN,NaN,20.6


* Problèmes rencontrés :
    - Présence de colonnes inutiles.
    - Lignes contenant des valeurs nulles.
    - Colonnes illisibles.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; **Nettoyage des données :**
* Sélection des colonnes utiles :

In [16]:
df2 = df2[['geo\\TIME_PERIOD','age','sex','isced11','2019']]
df2.sample(3)

,geo\TIME_PERIOD,age,sex,isced11,2019
2373,ES,Y16-24,F,ED3_4,9.0
3174,DE,Y25-64,F,ED3_4,NaN
4815,EA,Y16-24,T,ED5-8,NaN


* Renommage des colonnes : 

In [17]:
df2 = df2.rename(columns={
    'geo\\TIME_PERIOD':'Geo',
    'isced11':'Type_Education'
})
df2.head(3)

,Geo,age,sex,Type_Education,2019
0,AL,Y16-24,F,ED0-2,4.0
1,AT,Y16-24,F,ED0-2,17.9
2,BE,Y16-24,F,ED0-2,14.2


* Remplacement des codes pays par leur nom complet :

In [18]:
ds2_coutry_code = {
    "EU": "Union européenne (UE6-1958, UE9-1973, UE10-1981, UE12-1986, UE15-1995, UE25-2004, UE27-2007, UE28-2013, UE27-2020)",
    "EU27_2020": "Union européenne - 27 pays (à partir de 2020)",
    "EU28": "Union européenne - 28 pays (2013-2020)",
    "EU27_2007": "Union européenne - 27 pays (2007-2013)",
    "EA": "Zone euro (EA11-1999, EA12-2001, EA13-2007, EA15-2008, EA16-2009, EA17-2011, EA18-2014, EA19-2015, EA20-2023)",
  
    "RS": "Serbie",
    "TR": "Turquie",
    "XK": "Kosovo*"
}
df2['Geo'] = df2['Geo'].replace(ds2_coutry_code)

* Remplacement des codes de sexe par leur nom complet :

In [19]:
df2['sex'] = df2['sex'].replace({'M': 'Male', 'F': 'Female','T':'Total'})

* Remplacement des codes d'education :

In [20]:
# Suppression des lignes où 'Type_Education' contient 'TOTAL'
df2 = df2[~df2['Type_Education'].isin(['TOTAL'])]

ds2_education_code = {
    "ED0-2": "Niv_Primaire",
    "ED3_4": "Niv_Secondaire",
    "ED5-8": "Niv_Supérieur",
}

# Remplacer les codes dans 'Type_Education' par les descriptions complètes
df2['Type_Education'] = df2['Type_Education'].replace(ds2_education_code)
df2.sample(3)


,Geo,age,sex,Type_Education,2019
6139,Zone euro - 18 pays (2014),Y55-64,Female,Niv_Supérieur,41.2
6117,Pays-Bas,Y45-64,Total,Niv_Supérieur,30.4
1892,Lettonie,Y_GE16,Male,Niv_Primaire,43.0


* Remplissage des valeurs manquantes pour l'année 2019 :

In [ ]:
# Remplir les valeurs manquantes de l'année 2019 par les valeurs de l'année précédente (2018)
df2['2019'] = df2.groupby('Geo')['2019'].fillna(method='ffill')

# Garder uniquement les lignes avec des données valides pour 2019 en cas que toutes les années contient des valeur null
df2 = df2[df2['2019'].notna()]  

* Filtrer le DataFrame pour inclure uniquement les lignes avec 'TOTAL' pour sexe

In [22]:
df2 = df2[df2['sex'] =="Total"]

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; **Présentation des données final :**
* Affichage des données après nettoyage 

In [23]:
df2.head(7)

,Geo,age,sex,Type_Education,2019
90,Albanie,Y16-24,Total,Niv_Primaire,4.0
91,Autriche,Y16-24,Total,Niv_Primaire,19.3
92,Belgique,Y16-24,Total,Niv_Primaire,12.2
93,Bulgarie,Y16-24,Total,Niv_Primaire,3.1
94,Suisse,Y16-24,Total,Niv_Primaire,21.8
95,Chypre,Y16-24,Total,Niv_Primaire,10.9
97,Allemagne,Y16-24,Total,Niv_Primaire,18.7


* Pivotage des données :

In [24]:
pivot_df2 = df2.pivot_table(
    index=['Geo'],  
    columns='Type_Education',  
    values='2019',  
).reset_index()

pivot_df2 = pivot_df2.dropna()
# preserve 2 nombre aprés la virgule
pivot_df2 = pivot_df2.round(2)


pivot_df2.head(10)

Type_Education,Geo,Niv_Primaire,Niv_Secondaire,Niv_Supérieur
0,Albanie,32.55,26.91,32.13
1,Allemagne,49.12,43.37,38.51
2,Autriche,47.93,37.04,34.68
3,Belgique,34.11,25.28,20.81
4,Bulgarie,28.01,26.26,25.98
5,Chypre,54.96,43.17,38.99
6,Croatie,47.84,35.03,36.01
7,Danemark,37.14,31.89,30.82
8,Espagne,34.65,26.77,24.44
9,Estonie,55.50,44.73,45.14


**3.** **Personnes avec un problème de santé ou une maladie de longue durée**,  
   par sexe, âge et **degré d'urbanisation**.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; **Inspection des données :**
* Affichage des données bruts

In [26]:
df3 = eurostat.get_data_df('hlth_silc_19')
df3.sample(5)

,freq,deg_urb,age,sex,unit,geo\TIME_PERIOD,2003,2004,2005,2006,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
945,A,DEG1,Y45-54,M,PC,IE,NaN,NaN,26.5,26.5,...,26.8,28.6,30.6,40.9,31.2,36.2,29.1,27.3,32.3,36.7
5386,A,DEG3,Y65-74,T,PC,EU28,NaN,NaN,NaN,NaN,...,55.2,57.9,57.2,57.7,57.2,57.5,NaN,NaN,NaN,NaN
4977,A,DEG3,Y45-54,M,PC,MK,NaN,NaN,NaN,NaN,...,NaN,NaN,7.4,12.0,11.7,8.4,8.2,NaN,NaN,NaN
592,A,DEG1,Y25-29,T,PC,DE,NaN,NaN,17.9,20.9,...,22.6,23.4,22.6,25.4,26.5,24.7,NaN,NaN,20.3,18.7
1591,A,DEG1,Y_GE16,M,PC,SK,NaN,NaN,22.7,21.8,...,26.1,26.0,25.3,26.6,24.0,26.0,27.6,26.9,29.1,27.0


* Problèmes rencontrés :
    - Présence de colonnes inutiles.
    - Lignes contenant des valeurs nulles.
    - Colonnes illisibles.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; **Nettoyage des données :**
* Sélection des colonnes utiles : 

In [28]:
df3 = df3[['geo\\TIME_PERIOD','sex','age','deg_urb','2019']]
df3.head(5)

,geo\TIME_PERIOD,sex,age,deg_urb,2019
0,CH,F,TOTAL,DEG1,NaN
1,DE,F,TOTAL,DEG1,NaN
2,CH,M,TOTAL,DEG1,NaN
3,DE,M,TOTAL,DEG1,NaN
4,CH,T,TOTAL,DEG1,NaN


* Renommage des colonnes :

In [29]:
df3 = df3.rename(columns={
    'geo\\TIME_PERIOD':'Geo',
    'deg_urb':'Degré_urbanisation'
})
df3.head(2)

,Geo,sex,age,Degré_urbanisation,2019
0,CH,F,TOTAL,DEG1,NaN
1,DE,F,TOTAL,DEG1,NaN


* Remplacement des codes pays par leur nom complet :

In [30]:
ds3_coutry_code = {
    "EU": "Union européenne (UE6-1958, UE9-1973, UE10-1981, UE12-1986, UE15-1995, UE25-2004, UE27-2007, UE28-2013, UE27-2020)",
    "EU27_2020": "Union européenne - 27 pays (à partir de 2020)",
    "EU28": "Union européenne - 28 pays (2013-2020)",
   
    "AL": "Albanie",
    "RS": "Serbie"
}
df3['Geo'] = df3['Geo'].replace(ds3_coutry_code)

* Remplacement des codes de Degré d'urbanisation :

In [31]:
# Suppression des lignes où 'Activité_situation' contient 'TOTAL' 
df3 = df3[~df3['Degré_urbanisation'].isin(['TOTAL'])]

ds3_Degré_urbanisation_code = {
    "DEG1": "Villes",
    "DEG2": "Villes_moins_peuplées_et_banlieues",
    "DEG3": "Zones_rurales"
}

# Remplacer les codes dans 'Degré_urbanisation' par les descriptions complètes
df3['Degré_urbanisation'] = df3['Degré_urbanisation'].replace(ds3_Degré_urbanisation_code)

* Remplissage des valeurs manquantes pour l'année 2019 :

In [ ]:
# Remplir les valeurs manquantes de l'année 2019 par les valeurs de l'année précédente (2018)
df3['2019'] = df3.groupby('Geo')['2019'].fillna(method='ffill')

# Garder uniquement les lignes avec des données valides pour 2019 en cas que toutes les années contient des valeur null
df3 = df3[df3['2019'].notna()]  

* Remplacement des codes de sexe par leur nom complet :

In [33]:
df3['sex'] = df3['sex'].replace({'M': 'Male', 'F': 'Female','T':'Total'})

* Filtrer le DataFrame pour inclure uniquement les lignes avec 'TOTAL' pour sexe

In [34]:
df3 = df3[df3['sex'] =="Total"]

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; **Présentation des données final :**
* Affichage des données après nettoyage :

In [35]:
df3.sample(6)

,Geo,sex,age,Degré_urbanisation,2019
1236,France,Total,Y55-64,Villes,46.8
5383,"Union européenne (UE6-1958, UE9-1973, UE10-198...",Total,Y65-74,Zones_rurales,57.5
2479,Chypre,Total,Y16-64,Villes_moins_peuplées_et_banlieues,15.3
4868,Chypre,Total,Y35-44,Zones_rurales,27.7
234,Union européenne - 27 pays (à partir de 2020),Total,Y16-29,Villes,17.7
3486,Tchéquie,Total,Y75-84,Villes_moins_peuplées_et_banlieues,74.4


* Pivotage des données :

In [36]:
pivot_df3 = df3.pivot_table(
    index=['Geo','age'],  
    columns='Degré_urbanisation',  
    values='2019',  
).reset_index()

pivot_df3 = pivot_df3.dropna()
# preserve 2 nombre aprés la virgule
pivot_df3 = pivot_df3.round(2)

pivot_df3.sample(7)

Degré_urbanisation,Geo,age,Villes,Villes_moins_peuplées_et_banlieues,Zones_rurales
554,Union européenne - 27 pays (2007-2013),Y55-64,45.8,47.5,46.7
253,Islande,Y_GE65,50.9,44.7,38.9
157,Estonie,Y_GE65,77.9,78.3,77.6
428,Roumanie,Y_GE16,18.0,17.5,20.3
313,Luxembourg,Y55-64,21.6,38.6,32.4
235,Irlande,Y75-84,59.0,55.5,58.1
476,Slovénie,Y_GE16,35.9,35.4,36.0


**4.** **Personnes avec un problème de santé ou une maladie de longue durée**,  
   par sexe, âge et statut au **type de maladie et education**.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; **Inspection des données :**
* Affichage des données bruts

In [38]:
df4 = eurostat.get_data_df('hlth_ehis_cd1e')
df4.sample(6)

,freq,unit,hlth_pb,isced11,sex,age,geo\TIME_PERIOD,2014,2019
8207,A,PC,ARTHRO,ED3_4,T,Y15-64,TR,2.9,6.4
56064,A,PC,HRT_CHR_ATK,ED0-2,M,Y_GE65,EU27_2020,6.9,6.9
85109,A,PC,URIN,ED3_4,F,Y15-64,AT,1.8,1.7
866,A,PC,ALLGY,ED0-2,M,Y65-74,MT,8.9,4.7
77203,A,PC,NECK,TOTAL,M,Y45-54,AT,18.9,18.7
31726,A,PC,CIRRHOSIS,ED5-8,M,Y15-24,BE,0.0,0.0


In [39]:
df4 = df4[['geo\\TIME_PERIOD','age','sex','hlth_pb','isced11','2019']]
df4.head(3)

,geo\TIME_PERIOD,age,sex,hlth_pb,isced11,2019
0,AT,TOTAL,F,ALLGY,ED0-2,19.0
1,BE,TOTAL,F,ALLGY,ED0-2,18.8
2,BG,TOTAL,F,ALLGY,ED0-2,4.4


* Renommage des colonnes :

In [40]:
df4 = df4.rename(columns={
    'geo\\TIME_PERIOD':'Geo',
    'hlth_pb':'Problèmes_santé',
    'isced11':'Type_Education'
})
df4.head(1)

,Geo,age,sex,Problèmes_santé,Type_Education,2019
0,AT,TOTAL,F,ALLGY,ED0-2,19.0


* Remplacement des codes pays par leur nom complet :

In [41]:
ds4_coutry_code = {
    "EU27_2020": "Union européenne - 27 pays (à partir de 2020)",
    "EU28": "Union européenne - 28 pays (2013-2020)",
    "BE": "Belgique",
    "BG": "Bulgarie",
    "TR": "Turquie"
}

df4['Geo'] = df4['Geo'].replace(ds4_coutry_code)

* Remplacement des codes de sexe par leur nom complet :

In [42]:
df4['sex'] = df4['sex'].replace({'M': 'Male', 'F': 'Female','T':'Total'})

* Remplacement des codes d'education et type de malade :

In [43]:
ds4_maladie_code = {
    "ARTHRO": "Arthrose",
    "LW_CHR_BACK": "Dos_chronique",
    "NECK": "Cou_chronique",
    "ASTHMA": "Asthme",
    "CHR_LW_RES": "Resp_chronique",
    "HRT_CHR_ATK": "Crise_cardiaque",
    "CHRT_ANGPEC": "Mal_coronaire",
    "HBLPR": "HTA",
    "HBLLIP": "Lipides_haut",
    "STRK_CHR": "AVC",
    "CIRRHOSIS": "Cirrhose",
    "URIN": "Incontinence",
    "KIDNEY": "Reins",
    "DIAB": "Diabète",
    "ALLGY": "Allergie",
}

# Suppression des lignes où 'Type_Education' contient 'TOTAL'
df4 = df4[~df4['Type_Education'].isin(['TOTAL'])]

ds4_education_code = {
    "ED0-2": "Niv_Primaire",
    "ED3_4": "Niv_Secondaire",
    "ED5-8": "Niv_Supérieur",
}

# Remplacer les codes dans 'Problèmes_santé' par les descriptions complètes
df4['Problèmes_santé'] = df4['Problèmes_santé'].replace(ds4_maladie_code)

# Remplacer les codes dans 'Type_Education' par les descriptions complètes
df4['Type_Education'] = df4['Type_Education'].replace(ds4_education_code)

* Remplissage des valeurs manquantes pour l'année 2019 :

In [ ]:
# Remplir les valeurs manquantes de l'année 2019 par les valeurs de l'année précédente (2014)
df4['2019'] = df4.groupby('Geo')['2019'].fillna(method='ffill')

# Garder uniquement les lignes avec des données valides pour 2019 en cas que toutes les années contient des valeur null
df4 = df4[df4['2019'].notna()]  


* Filtrer le DataFrame pour inclure uniquement les lignes avec 'TOTAL' pour sexe

In [45]:
df4 = df4[df4['sex'] =="Total"]

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; **Présentation des données final :**
* Affichage des données après nettoyage :

In [46]:
df4.sample(7)

,Geo,age,sex,Problèmes_santé,Type_Education,2019
76293,Bulgarie,Y45-64,Total,Cou_chronique,Niv_Supérieur,4.5
4032,France,Y35-44,Total,Allergie,Niv_Supérieur,28.6
31133,Islande,Y65-74,Total,Cirrhose,Niv_Secondaire,0.9
70588,Estonie,Y45-54,Total,Dos_chronique,Niv_Supérieur,22.4
48449,Chypre,Y_GE75,Total,Lipides_haut,Niv_Supérieur,51.5
64839,Belgique,Y25-64,Total,Reins,Niv_Supérieur,0.2
13731,Croatie,TOTAL,Total,Asthme,Niv_Secondaire,3.7


* Pivotage de dataFrame df4 :

In [47]:
# Création d'un tableau croisé dynamique à partir du DataFrame 'df'.
pivot_df4 = df4.pivot_table(
    index=['Geo'],  
    columns='Problèmes_santé',  
    values='2019',  
).reset_index()

# Suppression des lignes contenant des valeurs manquantes (NaN) dans le DataFrame.
pivot_df4 = pivot_df4.dropna()

# Arrondi des valeurs numériques du DataFrame à deux décimales.
pivot_df4 = pivot_df4.round(2)

pivot_df4.head(6)

Problèmes_santé,Geo,AVC,Allergie,Arthrose,Asthme,Cirrhose,Cou_chronique,Crise_cardiaque,DPR_CHR,Diabète,Dos_chronique,HTA,Incontinence,Lipides_haut,Mal_coronaire,Reins,Resp_chronique
0,Allemagne,1.95,31.47,14.52,7.86,0.43,20.83,2.11,11.43,7.61,27.96,23.29,7.43,18.77,3.86,2.28,5.40
1,Autriche,1.53,22.73,13.28,4.40,0.34,18.00,1.65,7.83,5.91,24.61,20.47,4.39,20.60,3.23,2.49,4.55
2,Belgique,0.64,18.70,17.66,5.55,0.80,13.60,0.72,6.84,5.66,23.31,16.46,4.46,20.89,1.33,0.85,3.70
3,Bulgarie,2.32,4.10,5.31,2.13,0.28,3.92,1.43,2.70,6.16,9.30,26.34,2.50,2.92,6.56,4.26,3.09
4,Chypre,1.62,7.09,10.12,3.90,0.26,9.69,3.32,5.26,7.96,15.51,20.90,3.65,24.55,2.35,2.37,3.04
5,Croatie,2.29,17.87,9.11,4.54,0.82,25.51,2.62,9.47,8.75,34.44,27.77,10.45,19.02,6.82,5.71,5.10



## II. Blessures lors d'accidents *(DataSet Groupe)* : Hamza OUADOUD

**1. Personnes déclarant un accident entraînant des blessures** par sexe, âge et **degré d'urbanisation**.



&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; **Inspection des données :**
* Affichage des données bruts

In [68]:
df5 = eurostat.get_data_df('hlth_ehis_ac1u')
df5.sample(5)

,freq,accident,deg_urb,sex,age,unit,geo\TIME_PERIOD,2014,2019
24061,A,RD_TRF,DEG3,F,Y15-29,PC,LV,0.0,0.5
18424,A,LEIS,DEG3,T,Y15-64,PC,IS,8.2,5.7
6121,A,HOM,TOTAL,M,Y45-64,PC,LV,2.4,2.2
1580,A,HOM,DEG1,T,Y55-64,PC,DK,0.6,2.0
19337,A,LEIS,TOTAL,F,Y_GE65,PC,PL,0.7,1.2


* Problèmes rencontrés :
    - Présence de colonnes inutiles.
    - Lignes contenant des valeurs nulles.
    - Colonnes avec des types de données inappropriés.
    - Colonnes illisibles.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; **Nettoyage des données :**
* Sélection des colonnes utiles : 

In [70]:
df5 = df5[['geo\\TIME_PERIOD','age','sex','accident','deg_urb','2019']]
df5.head(3)

,geo\TIME_PERIOD,age,sex,accident,deg_urb,2019
0,AT,TOTAL,F,HOM,DEG1,6.6
1,BE,TOTAL,F,HOM,DEG1,5.2
2,BG,TOTAL,F,HOM,DEG1,2.0


* Renommage des colonnes :

In [71]:
df5 = df5.rename(columns={
    'geo\\TIME_PERIOD':'Geo',
    'deg_urb':'Degré_urbanisation',
    'accident':'Accident'
})
df5.head(0)

,Geo,age,sex,Accident,Degré_urbanisation,2019


* Remplacement des codes pays par leur nom complet :

In [72]:
ds5_coutry_code = {
    "EU27_2020": "Union européenne - 27 pays (à partir de 2020)",
   
    "TR": "Turquie"
}
df5['Geo'] = df5['Geo'].replace(ds5_coutry_code)

* Remplacement des codes de sexe par leur nom complet :

In [73]:
df5['sex'] = df5['sex'].replace({'M': 'Male', 'F': 'Female','T':'Total'})

* Remplacement des codes d'Accident et Degré urbanisation :


In [74]:
# Suppression des lignes où 'Degré_urbanisation' contient 'TOTAL'
df5 = df5[~df5['Degré_urbanisation'].isin(['TOTAL'])]

ds5_accident_code = {
    "RD_TRF": "Trafic routier",
    "HOM_LEIS": "Domicile et loisir",
    "HOM": "Domicile",
    "LEIS": "Loisir"
}

ds5_Degré_urbanisation_code = {
    "DEG1": "Villes",
    "DEG2": "Villes_moins_peuplées_et_banlieues",
    "DEG3": "Zones_rurales"
}

# Remplacer les codes dans 'Accident' par les descriptions complètes
df5['Accident'] = df5['Accident'].replace(ds5_accident_code)

# Remplacer les codes dans 'Degré_urbanisation' par les descriptions complètes
df5['Degré_urbanisation'] = df5['Degré_urbanisation'].replace(ds5_Degré_urbanisation_code)

* Remplissage des valeurs manquantes pour l'année 2019 :


In [ ]:
# Remplir les valeurs manquantes de l'année 2019 par les valeurs de l'année précédente (2014)
df5['2019'] = df5.groupby('Geo')['2019'].fillna(method='ffill')

# Garder uniquement les lignes avec des données valides pour 2019 en cas que toutes les années contient des valeur null
df5 = df5[df5['2019'].notna()]  

* Filtrer le DataFrame pour inclure uniquement les lignes avec 'TOTAL' pour sexe

In [76]:
df5 = df5[df5['sex'] =="Total"]

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; **Présentation des données final :**
* Affichage des données après nettoyage :

In [77]:
df5.sample(8)

,Geo,age,sex,Accident,Degré_urbanisation,2019
2984,Autriche,Y15-44,Total,Domicile,Villes_moins_peuplées_et_banlieues,4.9
2910,Roumanie,Y15-19,Total,Domicile,Villes_moins_peuplées_et_banlieues,1.4
11593,Portugal,Y15-64,Total,Domicile et loisir,Zones_rurales,3.0
23766,Slovénie,Y45-54,Total,Trafic routier,Villes_moins_peuplées_et_banlieues,2.1
1595,Lettonie,Y55-64,Total,Domicile,Villes,0.9
14883,Slovaquie,Y15-19,Total,Loisir,Villes,3.1
22030,Tchéquie,Y45-54,Total,Trafic routier,Villes,3.0
23924,Italie,Y_GE75,Total,Trafic routier,Villes_moins_peuplées_et_banlieues,1.3


* Pivotage des données :

In [78]:
# Création d'un tableau croisé dynamique à partir du DataFrame 'df'.
pivot_df5 = df5.pivot_table(
    index=['Geo','Degré_urbanisation'],  
    columns='Accident',  
    values='2019',  
).reset_index()

# Suppression des lignes contenant des valeurs manquantes (NaN) dans le DataFrame.
pivot_df5 = pivot_df5.dropna()

# Arrondi des valeurs numériques du DataFrame à deux décimales.
pivot_df5 = pivot_df5.round(2)

pivot_df5.head(6)

Accident,Geo,Degré_urbanisation,Domicile,Domicile et loisir,Loisir,Trafic routier
0,Allemagne,Villes,5.18,13.16,9.85,2.63
1,Allemagne,Villes_moins_peuplées_et_banlieues,5.53,13.70,10.07,2.24
2,Allemagne,Zones_rurales,5.22,12.21,8.41,2.14
3,Autriche,Villes,5.96,13.00,8.03,1.73
4,Autriche,Villes_moins_peuplées_et_banlieues,4.80,12.86,8.81,2.07
5,Autriche,Zones_rurales,5.27,11.82,7.22,1.64


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler


# Sélectionner les variables numériques
features = pivot_df.columns[1:]

# Liste des payes à exclure , je vais exclus ces payes car sa nom est tres longue
excluded_entities = [
    "European Union (EU6-1958, EU9-1973, EU10-1981, EU12-1986, EU15-1995, EU25-2004, EU27-2007, EU28-2013, EU27-2020)",
    "European Union - 27 countries (from 2020)",
    "European Union - 28 countries (2013-2020)",
    "European Union - 27 countries (2007-2013)",
    "Euro area (EA11-1999, EA12-2001, EA13-2007, EA15-2008, EA16-2009, EA17-2011, EA18-2014, EA19-2015, EA20-2023)",
    "Euro area – 20 countries (from 2023)",
    "Euro area - 19 countries (2015-2022)",
    "Euro area - 18 countries (2014)"
]

pivot_df = pivot_df[~pivot_df['Geo'].isin(excluded_entities)]

# Standardisation des données (ACP non normée car les données sont en pourcentage)
X = pivot_df[features].values

# Application de l'ACP
pca = PCA()
X_pca = pca.fit_transform(X)

# Matrice de covariance
cov_matrix = np.cov(X.T)

# # Visualisation de la matrice de covariance avec Seaborn
# plt.figure(figsize=(10, 8))
# sns.heatmap(cov_matrix, annot=True, fmt=".2f", cmap='coolwarm', xticklabels=features, yticklabels=features)
# plt.title('Matrice de Covariance')
# plt.show()

# print("Une grande covariance entre \"Chômeurs\" et \"Personnes_non_occupées\" reflète une forte interdépendance positive. En revanche, les faibles covariances impliquant \"Cluster\" montrent qu'il a peu de lien avec les autres variables.")
# # Inertie expliquée par chaque composant
# explained_variance = pca.explained_variance_ratio_
# plt.figure(figsize=(10, 6))
# plt.bar(range(1, len(explained_variance) + 1), explained_variance, alpha=0.5, align='center', label='Individual explained variance')
# plt.step(range(1, len(explained_variance) + 1), np.cumsum(explained_variance), where='mid', label='Cumulative explained variance')
# plt.ylabel('Explained variance ratio')
# plt.xlabel('Principal components')
# plt.legend(loc='best')
# plt.tight_layout()
# plt.show()

# # Graphe des pays avec leur nom
# plt.figure(figsize=(8, 8))
# for i in range(len(pivot_df)):
#     plt.text(X_pca[i, 0], X_pca[i, 1], pivot_df['Geo'].iloc[i], color='blue', ha='center', va='center')
# plt.xlabel('PC1')
# plt.ylabel('PC2')
# plt.title('Projection des Pays sur les deux premières Composantes Principales')
# plt.grid()
# plt.show()

# # Cercle de corrélation
# plt.figure(figsize=(10, 10))
# for i in range(len(features)):
#     plt.arrow(0, 0, pca.components_[0, i], pca.components_[1, i], color='r', alpha=0.5)
#     plt.text(pca.components_[0, i], pca.components_[1, i], features[i], color='g', ha='center', va='center')
# plt.xlim(-1, 1)
# plt.ylim(-1, 1)
# plt.xlabel('PC1')
# plt.ylabel('PC2')
# plt.title('Cercle de Corrélation des variables')
# plt.axhline(y=0, color='k', linestyle='--')
# plt.axvline(x=0, color='k', linestyle='--')
# plt.grid()
# plt.show()

In [ ]:
from scipy import stats

# Regroupement des âges en catégories
def categorize_age(age):
    if age == 'Y25-34':
        return 'Jeunes adultes (25-34)'
    elif age == 'Y55-64':
        return 'Adultes (55-64)'
    elif age == 'Y_GE75':
        return 'Seniors (75+)'
    else:
        return 'Population générale (16+)'

pivot_df3['age_category'] = pivot_df3['age'].apply(categorize_age)

# Préparation des données pour ANOVA
melted_df = pd.melt(pivot_df3, 
                    id_vars=['age_category', 'Geo'], 
                    value_vars=['Villes', 'Villes_moins_peuplées_et_banlieues', 'Zones_rurales'],
                    var_name='zone_type',
                    value_name='percentage')

# ANOVA à un facteur pour l'âge
print("=== Analyse ANOVA des maladies chroniques par groupe d'âge ===\n")


# ANOVA
age_groups = pivot_df3['age_category'].unique()
age_groups_data = [melted_df[melted_df['age_category'] == group]['percentage'] for group in age_groups]
f_stat, p_val = stats.f_oneway(*age_groups_data)


# Visualisations
plt.figure(figsize=(8, 8))

# Boxplot
plt.plot()
sns.boxplot(x='age_category', y='percentage', hue='zone_type', data=melted_df)
plt.title('Distribution des maladies chroniques\npar groupe d\'âge et type de zone')
plt.xticks(rotation=45)
plt.ylabel('Pourcentage')


plt.figure(figsize=(8, 8))
# Barplot
plt.plot()
sns.barplot(x='age_category', y='percentage', hue='zone_type', data=melted_df)
plt.title('Moyenne des maladies chroniques\npar groupe d\'âge et type de zone')
plt.xticks(rotation=45)
plt.ylabel('Pourcentage moyen')
plt.show()

# Test post-hoc de Fisher si ANOVA significative
if p_val < 0.05:
    print("\nTest post-hoc de Fisher (LSD):")
    from itertools import combinations
    
    for group1, group2 in combinations(age_groups, 2):
        data1 = melted_df[melted_df['age_category'] == group1]['percentage']
        data2 = melted_df[melted_df['age_category'] == group2]['percentage']
        
        t_stat, p_val_posthoc = stats.ttest_ind(data1, data2)

# Analyse des différences entre types de zones
print("\n=== Analyse des différences entre types de zones ===")
zone_types = ['Villes', 'Villes_moins_peuplées_et_banlieues', 'Zones_rurales']
f_stat_zones, p_val_zones = stats.f_oneway(pivot_df3['Villes'], 
                                          pivot_df3['Villes_moins_peuplées_et_banlieues'], 
                                          pivot_df3['Zones_rurales'])

# Titre principal
display(HTML("""<p><b>Interprétation :</b></p>
<p><b>Tendance par âge :</b></p>
<ul>
    <li>On observe une augmentation claire et progressive des maladies chroniques avec l'âge.</li>
    <li>Les jeunes adultes (25-34 ans) présentent le taux le plus bas (~15%).</li>
    <li>Les seniors (75+) ont le taux le plus élevé (~65-70%).</li>
    <li>Cette progression est cohérente dans tous les types de zones.</li>
</ul>
<p><b>Comparaison entre zones :</b></p>
<ul>
    <li>Les différences entre types de zones (villes, banlieues, zones rurales) sont relativement faibles.</li>
    <li>La variabilité (représentée par les barres d'erreur et les boîtes à moustaches) est plus importante chez les seniors.</li>
    <li>Les zones rurales montrent une légère tendance à des taux plus élevés chez les adultes (55-64 ans).</li>
</ul>
<p><b>Répartition :</b></p>
<ul>
    <li>Les box plots révèlent une dispersion plus importante des données pour les seniors.</li>
    <li>La distribution semble plus resserrée pour les jeunes adultes.</li>
    <li>Quelques valeurs atypiques sont observées, particulièrement dans la catégorie adultes (55-64 ans).</li>
</ul>
"""))
